<a href="https://colab.research.google.com/github/YanggWendy/FYP-Portfolio-Management-System/blob/main/FYP_Stock_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

FOLDERNAME = 'FYP Data/'
assert FOLDERNAME is not None, "[1]Enter the foldername."

import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

Mounted at /content/drive


In [8]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
PATH = '/content/drive/My Drive/FYP Data/' 

In [10]:
!ls "drive/My Drive/FYP Data/"

 price.csv  'stock data.csv'   test.csv


In [11]:
import yfinance as yf
import pandas as pd
from tqdm.notebook import tqdm
import statistics

In [12]:
aapl= yf.Ticker("aapl")
aapl_historical = aapl.history(period='max', interval="1d")
aapl_historical

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1980-12-12 00:00:00-05:00,0.099874,0.100308,0.099874,0.099874,469033600,0.0,0.0
1980-12-15 00:00:00-05:00,0.095098,0.095098,0.094663,0.094663,175884800,0.0,0.0
1980-12-16 00:00:00-05:00,0.088149,0.088149,0.087715,0.087715,105728000,0.0,0.0
1980-12-17 00:00:00-05:00,0.089886,0.090321,0.089886,0.089886,86441600,0.0,0.0
1980-12-18 00:00:00-05:00,0.092492,0.092927,0.092492,0.092492,73449600,0.0,0.0
...,...,...,...,...,...,...,...
2022-12-30 00:00:00-05:00,128.410004,129.949997,127.430000,129.929993,76960600,0.0,0.0
2023-01-03 00:00:00-05:00,130.279999,130.899994,124.169998,125.070000,112117500,0.0,0.0
2023-01-04 00:00:00-05:00,126.889999,128.660004,125.080002,126.360001,89113600,0.0,0.0


In [13]:
sp500url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
data_table = pd.read_html(sp500url)
data_table[0]

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...,...
498,YUM,Yum! Brands,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
499,ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
500,ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
501,ZION,Zions Bancorporation,reports,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


In [14]:
SP500_ticker = data_table[0]['Symbol'].tolist()
SP500_ticker

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ACN',
 'ATVI',
 'ADM',
 'ADBE',
 'ADP',
 'AAP',
 'AES',
 'AFL',
 'A',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AMD',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ACGL',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'AZO',
 'AVB',
 'AVY',
 'BKR',
 'BALL',
 'BAC',
 'BBWI',
 'BAX',
 'BDX',
 'WRB',
 'BRK.B',
 'BBY',
 'BIO',
 'TECH',
 'BIIB',
 'BLK',
 'BK',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF.B',
 'CHRW',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'CNC',
 'CNP',
 'CDAY',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CMA

In [15]:
data = yf.download(SP500_ticker, period='max', interval="1d", group_by='tickers')
data

[*********************100%***********************]  503 of 503 completed

2 Failed downloads:
- BRK.B: No timezone found, symbol may be delisted
- BF.B: 1d data not available for startTime=-2208994789 and endTime=1673189480. Only 100 years worth of day granularity data are allowed to be fetched per request.


LYV                                                         \
                 Open       High        Low      Close  Adj Close     Volume   
Date                                                                           
1962-01-02        NaN        NaN        NaN        NaN        NaN        NaN   
1962-01-03        NaN        NaN        NaN        NaN        NaN        NaN   
1962-01-04        NaN        NaN        NaN        NaN        NaN        NaN   
1962-01-05        NaN        NaN        NaN        NaN        NaN        NaN   
1962-01-08        NaN        NaN        NaN        NaN        NaN        NaN   
...               ...        ...        ...        ...        ...        ...   
2022-12-30  68.820000  69.800003  68.709999  69.739998  69.739998  1142700.0   
2023-01-03  70.809998  71.050003  68.519997  68.860001  68.860001  1296200.0   
2023-01-04  69.330002  71.629997  69.040001  71.010002  71.010002  1717700.0   
2023-01-05  70.599998  72.370003  69.900002  71.660004  71.660004  1727400.0   
2023-01-06  72.360001  73.629997  71.800003  72.379997  72.379997  1374700.0   

                  SBNY                                      ...         CRM  \
                  Open        High         Low       Close  ...         Low   
Date                                                        ...               
1962-01-02         NaN         NaN         NaN         NaN  ...         NaN   
1962-01-03         NaN         NaN         NaN         NaN  ...         NaN   
1962-01-04         NaN         NaN         NaN         NaN  ...         NaN   
1962-01-05         NaN         NaN         NaN         NaN  ...         NaN   
1962-01-08         NaN         NaN         NaN         NaN  ...         NaN   
...                ...         ...         ...         ...  ...         ...   
2022-12-30  113.980003  115.889999  113.339996  115.220001  ...  130.360001   
2023-01-03  117.139999  118.209999  112.559998  113.169998  ...  133.029999   
2023-01-04  115.800003  119.449997  114.260002  117.940002  ...  137.619995   
2023-01-05  110.320000  115.650002  109.440002  110.839996  ...  134.240005   
2023-01-06  109.489998  116.930000  106.809998  116.300003  ...  135.550003   

                                                      MKTX              \
                 Close   Adj Close      Volume        Open        High   
Date                                                                     
1962-01-02         NaN         NaN         NaN         NaN         NaN   
1962-01-03         NaN         NaN         NaN         NaN         NaN   
1962-01-04         NaN         NaN         NaN         NaN         NaN   
1962-01-05         NaN         NaN         NaN         NaN         NaN   
1962-01-08         NaN         NaN         NaN         NaN         NaN   
...                ...         ...         ...         ...         ...   
2022-12-30  132.589996  132.589996   5628700.0  279.700012  282.220001   
2023-01-03  134.779999  134.779999   8196400.0  281.869995  285.899994   
2023-01-04  139.589996  139.589996  14322600.0  288.109985  290.970001   
2023-01-05  136.339996  136.339996   8493500.0  284.899994  298.480011   
2023-01-06  140.509995  140.509995   9144100.0  300.410004  315.100006   

                                                          
                   Low       Close   Adj Close    Volume  
Date                                                      
1962-01-02         NaN         NaN         NaN       NaN  
1962-01-03         NaN         NaN         NaN       NaN  
1962-01-04         NaN         NaN         NaN       NaN  
1962-01-05         NaN         NaN         NaN       NaN  
1962-01-08         NaN         NaN         NaN       NaN  
...                ...         ...         ...       ...  
2022-12-30  275.829987  278.890015  278.890015  124800.0  
2023-01-03  277.790009  284.869995  284.869995  199700.0  
2023-01-04  282.369995  286.769989  286.769989  230500.0  
2023-01-05  284.470001  298.230011  298.23001

In [16]:
#data.to_csv("drive/My Drive/FYP Data/price.csv",index=True)

#Get Technical data 

In [17]:
data['AAP']

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1962-01-02,NaN,NaN,NaN,NaN,NaN,NaN
1962-01-03,NaN,NaN,NaN,NaN,NaN,NaN
1962-01-04,NaN,NaN,NaN,NaN,NaN,NaN
1962-01-05,NaN,NaN,NaN,NaN,NaN,NaN
1962-01-08,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2022-12-30,145.550003,147.449997,145.020004,147.029999,147.029999,1004700.0
2023-01-03,146.160004,151.660004,146.000000,151.539993,151.539993,1307400.0
2023-01-04,152.729996,154.500000,150.889999,151.889999,151.889999,834200.0


In [18]:
stock_data = data.reindex(index=data.index[::-1])
stock_data

LYV                                                         \
                 Open       High        Low      Close  Adj Close     Volume   
Date                                                                           
2023-01-06  72.360001  73.629997  71.800003  72.379997  72.379997  1374700.0   
2023-01-05  70.599998  72.370003  69.900002  71.660004  71.660004  1727400.0   
2023-01-04  69.330002  71.629997  69.040001  71.010002  71.010002  1717700.0   
2023-01-03  70.809998  71.050003  68.519997  68.860001  68.860001  1296200.0   
2022-12-30  68.820000  69.800003  68.709999  69.739998  69.739998  1142700.0   
...               ...        ...        ...        ...        ...        ...   
1962-01-08        NaN        NaN        NaN        NaN        NaN        NaN   
1962-01-05        NaN        NaN        NaN        NaN        NaN        NaN   
1962-01-04        NaN        NaN        NaN        NaN        NaN        NaN   
1962-01-03        NaN        NaN        NaN        NaN        NaN        NaN   
1962-01-02        NaN        NaN        NaN        NaN        NaN        NaN   

                  SBNY                                      ...         CRM  \
                  Open        High         Low       Close  ...         Low   
Date                                                        ...               
2023-01-06  109.489998  116.930000  106.809998  116.300003  ...  135.550003   
2023-01-05  110.320000  115.650002  109.440002  110.839996  ...  134.240005   
2023-01-04  115.800003  119.449997  114.260002  117.940002  ...  137.619995   
2023-01-03  117.139999  118.209999  112.559998  113.169998  ...  133.029999   
2022-12-30  113.980003  115.889999  113.339996  115.220001  ...  130.360001   
...                ...         ...         ...         ...  ...         ...   
1962-01-08         NaN         NaN         NaN         NaN  ...         NaN   
1962-01-05         NaN         NaN         NaN         NaN  ...         NaN   
1962-01-04         NaN         NaN         NaN         NaN  ...         NaN   
1962-01-03         NaN         NaN         NaN         NaN  ...         NaN   
1962-01-02         NaN         NaN         NaN         NaN  ...         NaN   

                                                      MKTX              \
                 Close   Adj Close      Volume        Open        High   
Date                                                                     
2023-01-06  140.509995  140.509995   9144100.0  300.410004  315.100006   
2023-01-05  136.339996  136.339996   8493500.0  284.899994  298.480011   
2023-01-04  139.589996  139.589996  14322600.0  288.109985  290.970001   
2023-01-03  134.779999  134.779999   8196400.0  281.869995  285.899994   
2022-12-30  132.589996  132.589996   5628700.0  279.700012  282.220001   
...                ...         ...         ...         ...         ...   
1962-01-08         NaN         NaN         NaN         NaN         NaN   
1962-01-05         NaN         NaN         NaN         NaN         NaN   
1962-01-04         NaN         NaN         NaN         NaN         NaN   
1962-01-03         NaN         NaN         NaN         NaN         NaN   
1962-01-02         NaN         NaN         NaN         NaN         NaN   

                                                          
                   Low       Close   Adj Close    Volume  
Date                                                      
2023-01-06  297.910004  314.000000  314.000000  463400.0  
2023-01-05  284.470001  298.230011  298.230011  466500.0  
2023-01-04  282.369995  286.769989  286.769989  230500.0  
2023-01-03  277.790009  284.869995  284.869995  199700.0  
2022-12-30  275.829987  278.890015  278.890015  124800.0  
...                ...         ...         ...       ...  
1962-01-08         NaN         NaN         NaN       NaN  
1962-01-05         NaN         NaN         NaN       NaN  
1962-01-04         NaN         NaN         NaN       NaN  
1962-01-03         NaN         NaN         Na

##12 day Volumn MA

In [19]:
#stock_data = pd.read_csv(("drive/My Drive/FYP Data/price.csv"),index_col = [0])

In [20]:
stock_data = data.reindex(index=data.index[::-1])
stock_data

LYV                                                         \
                 Open       High        Low      Close  Adj Close     Volume   
Date                                                                           
2023-01-06  72.360001  73.629997  71.800003  72.379997  72.379997  1374700.0   
2023-01-05  70.599998  72.370003  69.900002  71.660004  71.660004  1727400.0   
2023-01-04  69.330002  71.629997  69.040001  71.010002  71.010002  1717700.0   
2023-01-03  70.809998  71.050003  68.519997  68.860001  68.860001  1296200.0   
2022-12-30  68.820000  69.800003  68.709999  69.739998  69.739998  1142700.0   
...               ...        ...        ...        ...        ...        ...   
1962-01-08        NaN        NaN        NaN        NaN        NaN        NaN   
1962-01-05        NaN        NaN        NaN        NaN        NaN        NaN   
1962-01-04        NaN        NaN        NaN        NaN        NaN        NaN   
1962-01-03        NaN        NaN        NaN        NaN        NaN        NaN   
1962-01-02        NaN        NaN        NaN        NaN        NaN        NaN   

                  SBNY                                      ...         CRM  \
                  Open        High         Low       Close  ...         Low   
Date                                                        ...               
2023-01-06  109.489998  116.930000  106.809998  116.300003  ...  135.550003   
2023-01-05  110.320000  115.650002  109.440002  110.839996  ...  134.240005   
2023-01-04  115.800003  119.449997  114.260002  117.940002  ...  137.619995   
2023-01-03  117.139999  118.209999  112.559998  113.169998  ...  133.029999   
2022-12-30  113.980003  115.889999  113.339996  115.220001  ...  130.360001   
...                ...         ...         ...         ...  ...         ...   
1962-01-08         NaN         NaN         NaN         NaN  ...         NaN   
1962-01-05         NaN         NaN         NaN         NaN  ...         NaN   
1962-01-04         NaN         NaN         NaN         NaN  ...         NaN   
1962-01-03         NaN         NaN         NaN         NaN  ...         NaN   
1962-01-02         NaN         NaN         NaN         NaN  ...         NaN   

                                                      MKTX              \
                 Close   Adj Close      Volume        Open        High   
Date                                                                     
2023-01-06  140.509995  140.509995   9144100.0  300.410004  315.100006   
2023-01-05  136.339996  136.339996   8493500.0  284.899994  298.480011   
2023-01-04  139.589996  139.589996  14322600.0  288.109985  290.970001   
2023-01-03  134.779999  134.779999   8196400.0  281.869995  285.899994   
2022-12-30  132.589996  132.589996   5628700.0  279.700012  282.220001   
...                ...         ...         ...         ...         ...   
1962-01-08         NaN         NaN         NaN         NaN         NaN   
1962-01-05         NaN         NaN         NaN         NaN         NaN   
1962-01-04         NaN         NaN         NaN         NaN         NaN   
1962-01-03         NaN         NaN         NaN         NaN         NaN   
1962-01-02         NaN         NaN         NaN         NaN         NaN   

                                                          
                   Low       Close   Adj Close    Volume  
Date                                                      
2023-01-06  297.910004  314.000000  314.000000  463400.0  
2023-01-05  284.470001  298.230011  298.230011  466500.0  
2023-01-04  282.369995  286.769989  286.769989  230500.0  
2023-01-03  277.790009  284.869995  284.869995  199700.0  
2022-12-30  275.829987  278.890015  278.890015  124800.0  
...                ...         ...         ...       ...  
1962-01-08         NaN         NaN         NaN       NaN  
1962-01-05         NaN         NaN         NaN       NaN  
1962-01-04         NaN         NaN         NaN       NaN  
1962-01-03         NaN         NaN         Na

In [21]:
for ticker in tqdm(SP500_ticker):
  #VEMA12
  stock_data_test = stock_data
  stock_TV = stock_data_test[ticker]['Volume'].tolist()

  counter = 0
  TV_MA12 = 0
  TV_MA12_stock_TV = []
  for i in range(len(stock_TV)):
    if len(stock_TV) - i >= 12:
      for j in range(12):
        TV_MA12 += stock_TV[i+j]

      TV_MA12_stock_TV.append(TV_MA12/12)
      TV_MA12 = 0
  while len(TV_MA12_stock_TV) != len(stock_TV):
    TV_MA12_stock_TV.append(0)

  stock_data_test[ticker,"VEMA12"] = TV_MA12_stock_TV


  0%|          | 0/503 [00:00<?, ?it/s]

In [22]:
stock_data_test['FRT']

,Open,High,Low,Close,Adj Close,Volume,VEMA12
Date,,,,,,,
2023-01-06,103.779999,104.870003,103.510002,104.239998,104.239998,363900.0,400883.333333
2023-01-05,103.940002,104.680000,102.050003,102.720001,102.720001,444100.0,412033.333333
2023-01-04,103.349998,105.620003,103.349998,105.059998,105.059998,491600.0,487108.333333
2023-01-03,101.750000,102.910004,100.599998,102.660004,102.660004,635900.0,515283.333333
2022-12-30,100.830002,101.470001,99.949997,101.040001,101.040001,409200.0,517125.000000
...,...,...,...,...,...,...,...
1962-01-08,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
1962-01-05,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
1962-01-04,NaN,NaN,NaN,NaN,NaN,NaN,0.000000


In [23]:
#VSTD20
for ticker in tqdm(SP500_ticker):
  stock_TV = stock_data[ticker]['Volume'].tolist()

  counter = 0
  VSTD20 = []
  VSTD20_stock_TV = []
  for i in range(len(stock_TV)):
    if len(stock_TV) - i >= 20:
      for j in range(20):
        VSTD20.append(stock_TV[i+j])

      VSTD20_stock_TV.append(statistics.stdev(VSTD20))
      VSTD20 = []
  #print(len(VSTD20_stock_TV))
  while len(VSTD20_stock_TV) != len(stock_TV):
    VSTD20_stock_TV.append(0)

  stock_data_test[ticker,"VSTD20"] = VSTD20_stock_TV


  0%|          | 0/503 [00:00<?, ?it/s]

In [24]:
#V20
for ticker in tqdm(SP500_ticker):
  stock_Open = stock_data[ticker]['Open'].tolist()
  stock_High = stock_data[ticker]['High'].tolist()
  stock_Low = stock_data[ticker]['Low'].tolist()
  stock_Close = stock_data[ticker]['Close'].tolist()

  V20 = []
  V20_temp = []
  V20_AR = []


  for i in range(len(stock_Open)):
    if len(stock_Open) - i >= 252:
        V20_AR.append((stock_Close[i] - stock_Close[i+ 251])/stock_Close[i+ 251])

  while len(V20_AR) != len(stock_Open):
    V20_AR.append(0)

  for i in range(len(stock_Open)):
    if len(stock_Open) - i >= 20:
      for j in range(20):
        V20_temp.append(V20_AR[i+j])
      V20.append(statistics.stdev(V20_temp))
      V20_temp = []
  while len(V20) != len(stock_Open):
    V20.append(0)
  stock_data_test[ticker,"V20"] = V20

  0%|          | 0/503 [00:00<?, ?it/s]

In [25]:
#AR
for ticker in tqdm(SP500_ticker):
  stock_Open = stock_data[ticker]['Open'].tolist()
  stock_High = stock_data[ticker]['High'].tolist()
  stock_Low = stock_data[ticker]['Low'].tolist()
  stock_Close = stock_data[ticker]['Close'].tolist()

  AR = []
  AR_HOGap = []
  AR_OLGap = []

  for i in range(len(stock_Open)):
    if len(stock_Open) - i >= 26:
      for j in range(26):
        AR_HOGap.append(stock_High[i+j]-stock_Open[i+j])
        AR_OLGap.append(stock_Open[i+j]-stock_Low[i+j])
      if sum(AR_OLGap) == 0:
        break
      AR.append(sum(AR_HOGap)*100/sum(AR_OLGap))
      AR_HOGap = []
      AR_OLGap = []

  while len(AR) != len(stock_Open):
    AR.append(0)
  stock_data[ticker,"AR"] = AR

  0%|          | 0/503 [00:00<?, ?it/s]

In [26]:
#BR
for ticker in tqdm(SP500_ticker):
  stock_Open = stock_data[ticker]['Open'].tolist()
  stock_High = stock_data[ticker]['High'].tolist()
  stock_Low = stock_data[ticker]['Low'].tolist()
  stock_Close = stock_data[ticker]['Close'].tolist()

  BR = []
  BR_HCGap = []
  BR_CLGap = []

  for i in range(len(stock_Open)):
    if len(stock_Open) - i > 26:
      for j in range(26):
        BR_HCGap.append(stock_High[i+j]-stock_Close[i+j+1])
        BR_CLGap.append(stock_Close[i+j+1]-stock_Low[i+j])
      if sum(BR_CLGap) == 0:
        break
      BR.append(sum(BR_HCGap)/sum(BR_CLGap))
      BR_HCGap = []
      BR_CLGap = []

  while len(BR) != len(stock_Open):
    BR.append(0)
  stock_data[ticker,"BR"] = BR

  0%|          | 0/503 [00:00<?, ?it/s]

In [27]:
#AU Aroon up
for ticker in tqdm(SP500_ticker):
  stock_Open = stock_data[ticker]['Open'].tolist()
  stock_High = stock_data[ticker]['High'].tolist()
  stock_Low = stock_data[ticker]['Low'].tolist()
  stock_Close = stock_data[ticker]['Close'].tolist()

  AU = []
  AU_period = 25
  AU_High = []
  AU_High_index = 0

  for i in range(len(stock_Open)):
    if len(stock_Open) - i > 25:
      for j in range(25):
        AU_High.append(stock_High[1+i+j])

      AU_High_index = AU_High.index(max(AU_High))
      AU.append((AU_period - AU_High_index)/AU_period)
      AU_High = []
      AU_High_index = 0

  while len(AU) != len(stock_Open):
    AU.append(0)
  stock_data[ticker,"AU"] = AU

  0%|          | 0/503 [00:00<?, ?it/s]

In [28]:
#AD Aroon down
for ticker in tqdm(SP500_ticker):
  stock_Open = stock_data[ticker]['Open'].tolist()
  stock_High = stock_data[ticker]['High'].tolist()
  stock_Low = stock_data[ticker]['Low'].tolist()
  stock_Close = stock_data[ticker]['Close'].tolist()

  AD = []
  AD_period = 25
  AD_LOW = []
  AD_LOW_index = 0

  for i in range(len(stock_Open)):
    if len(stock_Open) - i > 25:
      for j in range(25):
        AD_LOW.append(stock_Low[1+i+j])

      AD_LOW_index = AD_LOW.index(min(AD_LOW))
      AD.append((AD_period - AD_LOW_index)/AD_period)
      AD_LOW = []
      AD_LOW_index = 0

  while len(AD) != len(stock_Open):
    AD.append(0)
  stock_data[ticker,"AD"] = AD
stock_data.to_csv("drive/My Drive/FYP Data/stock data.csv",index=False)

  0%|          | 0/503 [00:00<?, ?it/s]

In [29]:
#VOL10
stock_data['CTLT']

,Open,High,Low,Close,Adj Close,Volume,VEMA12,VSTD20,V20,AR,BR,AU,AD
Date,,,,,,,,,,,,,
2023-01-06,46.500000,47.740002,44.639999,47.660000,47.660000,2934600.0,1.981542e+06,1.413657e+06,0.018807,89.665250,0.851277,0.12,0.56
2023-01-05,45.630001,46.439999,45.250000,46.169998,46.169998,2606500.0,1.988250e+06,1.413367e+06,0.015732,95.290325,0.899025,0.16,0.60
2023-01-04,46.070000,47.110001,45.740002,46.450001,46.450001,2756800.0,2.280967e+06,1.415680e+06,0.014891,116.484365,1.147535,0.20,0.04
2023-01-03,46.000000,46.730000,44.494999,45.450001,45.450001,2053900.0,2.250342e+06,1.415206e+06,0.019105,112.135633,1.033790,0.24,0.08
2022-12-30,44.580002,45.500000,44.279999,45.009998,45.009998,2257100.0,2.262958e+06,1.453620e+06,0.023941,116.071067,1.008532,0.28,0.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1962-01-08,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.00,0.00
1962-01-05,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.00,0.00
1962-01-04,NaN,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.00,0.00


In [31]:
stock_data

LYV                                                         \
                 Open       High        Low      Close  Adj Close     Volume   
Date                                                                           
2023-01-06  72.360001  73.629997  71.800003  72.379997  72.379997  1374700.0   
2023-01-05  70.599998  72.370003  69.900002  71.660004  71.660004  1727400.0   
2023-01-04  69.330002  71.629997  69.040001  71.010002  71.010002  1717700.0   
2023-01-03  70.809998  71.050003  68.519997  68.860001  68.860001  1296200.0   
2022-12-30  68.820000  69.800003  68.709999  69.739998  69.739998  1142700.0   
...               ...        ...        ...        ...        ...        ...   
1962-01-08        NaN        NaN        NaN        NaN        NaN        NaN   
1962-01-05        NaN        NaN        NaN        NaN        NaN        NaN   
1962-01-04        NaN        NaN        NaN        NaN        NaN        NaN   
1962-01-03        NaN        NaN        NaN        NaN        NaN        NaN   
1962-01-02        NaN        NaN        NaN        NaN        NaN        NaN   

                  SBNY                                      ...   WTW   GWW  \
                  Open        High         Low       Close  ...    AD    AD   
Date                                                        ...               
2023-01-06  109.489998  116.930000  106.809998  116.300003  ...  0.04  1.00   
2023-01-05  110.320000  115.650002  109.440002  110.839996  ...  0.08  1.00   
2023-01-04  115.800003  119.449997  114.260002  117.940002  ...  0.12  1.00   
2023-01-03  117.139999  118.209999  112.559998  113.169998  ...  0.16  1.00   
2022-12-30  113.980003  115.889999  113.339996  115.220001  ...  0.20  0.68   
...                ...         ...         ...         ...  ...   ...   ...   
1962-01-08         NaN         NaN         NaN         NaN  ...  0.00  0.00   
1962-01-05         NaN         NaN         NaN         NaN  ...  0.00  0.00   
1962-01-04         NaN         NaN         NaN         NaN  ...  0.00  0.00   
1962-01-03         NaN         NaN         NaN         NaN  ...  0.00  0.00   
1962-01-02         NaN         NaN         NaN         NaN  ...  0.00  0.00   

            WYNN   XEL   XYL   YUM  ZBRA   ZBH  ZION   ZTS  
              AD    AD    AD    AD    AD    AD    AD    AD  
Date                                                        
2023-01-06  0.84  0.16  1.00  0.92  0.68  0.04  0.56  0.60  
2023-01-05  0.88  0.04  0.72  0.04  0.72  0.04  0.60  0.64  
2023-01-04  0.04  0.08  0.76  0.08  0.76  0.08  0.64  0.68  
2023-01-03  0.04  0.12  0.80  0.04  0.80  0.12  0.68  0.72  
2022-12-30  0.08  0.16  0.84  0.04  0.84  0.04  0.72  0.76  
...          ...   ...   ...   ...   ...   ...   ...   ...  
1962-01-08  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  
1962-01-05  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  
1962-01-04  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  
1962-01-03  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  
1962-01-02  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00  

[15360 rows x 6539 columns]

In [41]:
new_stock_data = stock_data['1990-01-02':'2023-01-06']
new_stock_data

LYV                                                         \
                 Open       High        Low      Close  Adj Close     Volume   
Date                                                                           
2023-01-06  72.360001  73.629997  71.800003  72.379997  72.379997  1374700.0   
2023-01-05  70.599998  72.370003  69.900002  71.660004  71.660004  1727400.0   
2023-01-04  69.330002  71.629997  69.040001  71.010002  71.010002  1717700.0   
2023-01-03  70.809998  71.050003  68.519997  68.860001  68.860001  1296200.0   
2022-12-30  68.820000  69.800003  68.709999  69.739998  69.739998  1142700.0   
...               ...        ...        ...        ...        ...        ...   
1990-01-09        NaN        NaN        NaN        NaN        NaN        NaN   
1990-01-08        NaN        NaN        NaN        NaN        NaN        NaN   
1990-01-05        NaN        NaN        NaN        NaN        NaN        NaN   
1990-01-04        NaN        NaN        NaN        NaN        NaN        NaN   
1990-01-03        NaN        NaN        NaN        NaN        NaN        NaN   

                  SBNY                                      ...   WTW   GWW  \
                  Open        High         Low       Close  ...    AD    AD   
Date                                                        ...               
2023-01-06  109.489998  116.930000  106.809998  116.300003  ...  0.04  1.00   
2023-01-05  110.320000  115.650002  109.440002  110.839996  ...  0.08  1.00   
2023-01-04  115.800003  119.449997  114.260002  117.940002  ...  0.12  1.00   
2023-01-03  117.139999  118.209999  112.559998  113.169998  ...  0.16  1.00   
2022-12-30  113.980003  115.889999  113.339996  115.220001  ...  0.20  0.68   
...                ...         ...         ...         ...  ...   ...   ...   
1990-01-09         NaN         NaN         NaN         NaN  ...  1.00  0.60   
1990-01-08         NaN         NaN         NaN         NaN  ...  1.00  0.64   
1990-01-05         NaN         NaN         NaN         NaN  ...  1.00  0.68   
1990-01-04         NaN         NaN         NaN         NaN  ...  1.00  0.04   
1990-01-03         NaN         NaN         NaN         NaN  ...  1.00  0.04   

            WYNN   XEL   XYL   YUM  ZBRA   ZBH  ZION   ZTS  
              AD    AD    AD    AD    AD    AD    AD    AD  
Date                                                        
2023-01-06  0.84  0.16  1.00  0.92  0.68  0.04  0.56  0.60  
2023-01-05  0.88  0.04  0.72  0.04  0.72  0.04  0.60  0.64  
2023-01-04  0.04  0.08  0.76  0.08  0.76  0.08  0.64  0.68  
2023-01-03  0.04  0.12  0.80  0.04  0.80  0.12  0.68  0.72  
2022-12-30  0.08  0.16  0.84  0.04  0.84  0.04  0.72  0.76  
...          ...   ...   ...   ...   ...   ...   ...   ...  
1990-01-09  1.00  0.04  1.00  1.00  1.00  1.00  0.80  1.00  
1990-01-08  1.00  0.04  1.00  1.00  1.00  1.00  0.84  1.00  
1990-01-05  1.00  0.04  1.00  1.00  1.00  1.00  0.88  1.00  
1990-01-04  1.00  0.08  1.00  1.00  1.00  1.00  0.92  1.00  
1990-01-03  1.00  0.12  1.00  1.00  1.00  1.00  0.96  1.00  

[8318 rows x 6539 columns]

In [52]:
new_stock_data.to_csv("drive/My Drive/FYP Data/stock data_1.csv",index=True, header=True )

In [54]:
new_stock_data.to_csv("drive/My Drive/FYP Data/stock data_2.csv",index=True, header=False )

In [53]:
stock_data_1 = pd.read_csv("drive/My Drive/FYP Data/stock data_1.csv")
stock_data_1

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252

,Unnamed: 0,LYV,LYV.1,LYV.2,LYV.3,LYV.4,LYV.5,SBNY,SBNY.1,SBNY.2,...,WTW.12,GWW.12,WYNN.12,XEL.12,XYL.12,YUM.12,ZBRA.12,ZBH.12,ZION.12,ZTS.12
0,NaN,Open,High,Low,Close,Adj Close,Volume,Open,High,Low,...,AD,AD,AD,AD,AD,AD,AD,AD,AD,AD
1,Date,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-01-06,72.36000061035156,73.62999725341797,71.80000305175781,72.37999725341797,72.37999725341797,1374700.0,109.48999786376953,116.93000030517578,106.80999755859375,...,0.04,1.0,0.84,0.16,1.0,0.92,0.68,0.04,0.56,0.6
3,2023-01-05,70.5999984741211,72.37000274658203,69.9000015258789,71.66000366210938,71.66000366210938,1727400.0,110.31999969482422,115.6500015258789,109.44000244140625,...,0.08,1.0,0.88,0.04,0.72,0.04,0.72,0.04,0.6,0.64
4,2023-01-04,69.33000183105469,71.62999725341797,69.04000091552734,71.01000213623047,71.01000213623047,1717700.0,115.80000305175781,119.44999694824219,114.26000213623047,...,0.12,1.0,0.04,0.08,0.76,0.08,0.76,0.08,0.64,0.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8315,1990-01-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.6,1.0,0.04,1.0,1.0,1.0,1.0,0.8,1.0
8316,1990-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.64,1.0,0.04,1.0,1.0,1.0,1.0,0.84,1.0
8317,1990-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.68,1.0,0.04,1.0,1.0,1.0,1.0,0.88,1.0
8318,1990-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,0.04,1.0,0.08,1.0,1.0,1.0,1.0,0.92,1.0


In [1]:
stock_data_2 = pd.read_csv("drive/My Drive/FYP Data/stock data_2.csv")
stock_data_2

NameError: ignored

#Fundamental Data

In [ ]:
!pip install fundamentalanalysis

In [ ]:
import fundamentalanalysis as fa
ticker = "CTLT"
api_key = "128c9cc9f6d57cb1f0c5503f2e1e69be"

In [ ]:
import fundamentalanalysis as fa

ticker = "MSFT"
api_key = "128c9cc9f6d57cb1f0c5503f2e1e69be"

# Obtain DCFs over time
dcf_quarterly = fa.discounted_cash_flow(ticker, api_key, period="quarter")

# Collect the Balance Sheet statements
balance_sheet_quarterly = fa.balance_sheet_statement(ticker, api_key, period="quarter")

# Collect the Income Statements
income_statement_quarterly = fa.income_statement(ticker, api_key, period="quarter")

# Collect the Cash Flow Statements
cash_flow_statement_quarterly = fa.cash_flow_statement(ticker, api_key, period="quarter")

# Show Key Metrics
key_metrics_quarterly = fa.key_metrics(ticker, api_key, period="quarter")

# Show a large set of in-depth ratios
financial_ratios_quarterly = fa.financial_ratios(ticker, api_key, period="quarter")

# Show the growth of the company
growth_quarterly = fa.financial_statement_growth(ticker, api_key, period="quarter")

In [ ]:
# Show Key Metrics
key_metrics_quarterly = fa.key_metrics(ticker, api_key, period="quarter")
key_metrics_quarterly

,2022-09,2022-06,2022-03,2021-12,2021-09,2021-06,2021-03,2020-12,2020-09,2020-06,...,2016-12,2016-09,2016-06,2016-03,2015-12,2015-09,2015-06,2015-03,2014-12,2014-09
period,Q1,Q4,Q3,Q2,Q1,Q4,Q3,Q2,Q1,Q4,...,Q2,Q1,Q4,Q3,Q2,Q1,Q4,Q3,Q2,Q1
revenuePerShare,5.677778,7.294444,7.072222,6.954286,5.994152,6.948538,6.177713,5.451439,5.153942,6.0627,...,3.873018,3.542717,4.264423,3.509337,3.644499,3.390694,4.082128,3.594841,3.626796,3.861231
netIncomePerShare,0.0,1.044444,0.783333,0.554286,0.54386,1.066667,1.359531,0.529103,0.50217,0.986564,...,0.139323,0.036853,0.495994,0.078519,0.245958,0.072944,1.23,0.253555,0.37,-0.18
operatingCashFlowPerShare,-0.511111,0.383333,0.766667,0.394286,0.953216,0.78538,0.439883,0.442914,0.912315,1.104926,...,0.381937,0.386959,0.271635,0.399808,0.21311,0.359911,0.617001,0.759054,0.319871,-0.367385
freeCashFlowPerShare,-1.338889,-0.922222,-0.055556,-0.308571,0.052632,-0.319298,-0.487977,-0.690109,0.000609,0.066539,...,0.170551,0.165038,0.016827,0.1971,-0.181864,0.093785,0.358517,0.458008,0.000796,-0.655385
cashPerShare,1.916667,2.988889,4.888889,5.228571,11.807018,5.654971,6.233431,4.986379,6.136951,6.098528,...,2.048208,1.120819,1.054487,1.072832,0.925346,1.213596,1.210794,0.93453,0.669183,0.583385
bookValuePerShare,26.111111,26.638889,25.733333,25.605714,25.614035,24.994152,23.883871,22.831045,22.419067,22.427383,...,4.708155,4.92311,5.095353,4.830546,4.866276,4.78705,5.07365,3.768707,3.925183,4.650462
tangibleBookValuePerShare,4.188889,4.05,2.905556,1.845714,6.175439,5.48538,4.23695,2.978899,1.937984,0.934741,...,-5.640178,-5.904528,-5.245192,-5.87854,-6.028766,-6.324485,-6.371672,-7.609054,-7.819333,-8.453538
shareholdersEquityPerShare,26.111111,26.638889,25.733333,25.605714,25.614035,24.994152,23.883871,22.831045,22.419067,22.427383,...,4.708155,4.92311,5.095353,4.830546,4.866276,4.78705,5.07365,3.768707,3.925183,4.650462
interestDebtPerShare,23.605556,23.6,23.516667,24.377143,25.403509,19.233333,19.147214,18.444994,18.794793,19.440179,...,16.392067,15.697872,15.082532,15.16306,15.182076,15.282971,15.288972,15.332409,15.580581,17.097231


In [ ]:
test = key_metrics_quarterly.T.reset_index()

In [ ]:
companies = fa.available_companies(api_key)
companies

,name,price,exchange,exchangeShortName,type
symbol,,,,,
6172.TW,Billionton Systems Inc.,20.1000,Taiwan,TAI,stock
FSIDX,Fidelity Strategic Dividend & Income Fund,15.6200,Nasdaq,NASDAQ,fund
VGRO.TO,Vanguard Growth ETF Portfolio,28.8900,Toronto Stock Exchange,TSX,etf
HYLE.DE,iShares Global High Yield Corp Bond UCITS ETF,4.3068,XETRA,XETRA,etf
DODIX,Dodge & Cox Income Fund - Class I,12.4300,Nasdaq,NASDAQ,NaN
...,...,...,...,...,...
TRUS.JK,PT Trust Finance Indonesia Tbk,426.0000,Jakarta Stock Exchange,JKT,trust
BLUT3.SA,Blue Tech Solutions E.Q.I. S.A.,5.1400,São Paulo,EURONEXT,stock
600346.SS,"Hengli Petrochemical Co.,Ltd.",16.0500,Shanghai,SHH,stock


In [ ]:
for ticker in tqdm(SP500_ticker):

In [26]:
#stock_data = pd.read_csv(("drive/My Drive/FYP Data/price.csv"),index_col = [0])
SP500 = pd.read_csv(("drive/My Drive/FYP Data/stock data.csv"),index_col = [0])
SP500

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,2

,CTLT.1,CTLT.2,CTLT.3,CTLT.4,CTLT.5,FRT,FRT.1,FRT.2,FRT.3,FRT.4,...,WTW.12,GWW.12,WYNN.12,XEL.12,XYL.12,YUM.12,ZBRA.12,ZBH.12,ZION.12,ZTS.12
CTLT,,,,,,,,,,,,,,,,,,,,,
Open,High,Low,Close,Adj Close,Volume,Open,High,Low,Close,Adj Close,...,AU,AU,AU,AU,AU,AU,AU,AU,AU,AU
69.69000244140625,71.30999755859375,70.94000244140625,71.30000305175781,71.30000305175781,17530.0,93.27999877929688,97.44000244140625,96.88529968261719,97.2699966430664,97.2699966430664,...,0.96,0.04,0.48,0.04,1.0,0.04,0.04,0.04,0.04,0.04
69.69000244140625,72.6500015258789,69.37000274658203,71.36000061035156,71.36000061035156,1294600.0,93.27999877929688,96.87999725341797,93.27999877929688,96.6500015258789,96.6500015258789,...,1.0,0.08,0.52,0.08,0.08,0.08,0.08,0.08,0.08,0.04
67.62999725341797,69.31999969482422,66.55000305175781,69.0,69.0,1462100.0,92.80000305175781,93.12000274658203,91.62999725341797,92.9000015258789,92.9000015258789,...,0.52,0.12,0.56,0.12,0.12,0.12,0.12,0.04,0.12,0.04
68.58999633789062,68.94499969482422,66.11000061035156,67.62999725341797,67.62999725341797,2774500.0,90.62000274658203,92.2699966430664,89.70999908447266,92.2699966430664,92.2699966430664,...,0.56,0.16,0.6,0.16,0.16,0.16,0.16,0.04,0.16,0.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
SP500

,CTLT,CTLT.1,CTLT.2,CTLT.3,CTLT.4,CTLT.5,FRT,FRT.1,FRT.2,FRT.3,...,WTW.12,GWW.12,WYNN.12,XEL.12,XYL.12,YUM.12,ZBRA.12,ZBH.12,ZION.12,ZTS.12
0,Open,High,Low,Close,Adj Close,Volume,Open,High,Low,Close,...,AU,AU,AU,AU,AU,AU,AU,AU,AU,AU
1,69.69000244140625,71.30999755859375,70.94000244140625,71.30000305175781,71.30000305175781,17530.0,93.27999877929688,97.44000244140625,96.88529968261719,97.2699966430664,...,0.96,0.04,0.48,0.04,1.0,0.04,0.04,0.04,0.04,0.04
2,69.69000244140625,72.6500015258789,69.37000274658203,71.36000061035156,71.36000061035156,1294600.0,93.27999877929688,96.87999725341797,93.27999877929688,96.6500015258789,...,1.0,0.08,0.52,0.08,0.08,0.08,0.08,0.08,0.08,0.04
3,67.62999725341797,69.31999969482422,66.55000305175781,69.0,69.0,1462100.0,92.80000305175781,93.12000274658203,91.62999725341797,92.9000015258789,...,0.52,0.12,0.56,0.12,0.12,0.12,0.12,0.04,0.12,0.04
4,68.58999633789062,68.94499969482422,66.11000061035156,67.62999725341797,67.62999725341797,2774500.0,90.62000274658203,92.2699966430664,89.70999908447266,92.2699966430664,...,0.56,0.16,0.6,0.16,0.16,0.16,0.16,0.04,0.16,0.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15308,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15310,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
SP500['CTLT']

0                     Open
1        69.69000244140625
2        69.69000244140625
3        67.62999725341797
4        68.58999633789062
               ...        
15307                  NaN
15308                  NaN
15309                  NaN
15310                  NaN
15311                  NaN
Name: CTLT, Length: 15312, dtype: object